In [1]:
import re
import pandas as pd

In [2]:
with open('data/LocalSP98.txt', 'r', errors='ignore') as f:
    data = f.readlines()

In [3]:
# removendo linhas em branco
data = list(filter(lambda linha: linha != '\n', data))

In [4]:
# removendo cabecalho
data = list(filter(lambda linha: not linha.startswith(' Tribunal Regional Eleitoral - SP'), data))
data = list(filter(lambda linha: not linha.startswith(' Sistema de Alistamento Eleitoral'), data))
data = list(filter(lambda linha: not linha.startswith(' Locais de Votacao/Secoes (Ordenado por ZONA)'), data))
data = list(filter(lambda linha: linha != ' ------------------------------------------------------------------------------\n', data))

In [5]:
# removendo quebra de linha e espaco no comeco
data = list(map(lambda linha: linha[1:-1], data))

In [7]:
# data

In [8]:
# Iterando sobre cada linha

resultado = []
padrao_cep = re.compile("[0-9]{5}[-]{0,1}[0-9]{3}")
padrao_secao_eleitores = re.compile("[0-9]{4}[/][0-9]{4}")
valida_secoes, valida_eleitores = [], []

for i in data:
    if i.startswith('Zona:'):
        zona = i[-3:]

    if i.startswith('Municipio'):
        cod_municipio = i.split(' - ')[0].split(': ')[-1]
        municipio = i.split(' - ')[-1]

    if i.startswith('Local'):
        cod_local = i.split(': ')[-1].split(' - ')[0]
        local = i.split(': ')[-1].split(' - ')[1]
    
    if i.startswith('Endereco  :'):
        endereco = i.split(':')[-1].lstrip()

    if i.startswith('Bairro    :'):
        bairro = i.split(':')[-1].lstrip()

    if i.startswith('Cep       :'):
        cep = padrao_cep.search(i).group()

    if i.startswith('Secao/Eleitores:') or i.startswith('                '):
        secao_eleitores = padrao_secao_eleitores.findall(i)
        for s in secao_eleitores:
            secao = s.split('/')[0]
            eleitores = int(s.split('/')[1])
            resultado.append([cod_municipio, municipio, cod_local, local, endereco, bairro, cep, zona, secao, eleitores])
        # break

    if i.startswith('Total Secoes/Eleitores:'):
        valida_secoes.append(int(i.split(' /')[0].split(':')[-1]))
        valida_eleitores.append(int(i.split(' /')[-1].replace('.', '')))
        # resultado.append([cod_municipio, municipio, cod_local, local, endereco, bairro, cep, secao, eleitores])
        # break

    


In [10]:
df_locais_votacao = pd.DataFrame(resultado, columns=['cod_municipio', 'municipio', 'cod_local', 'local', 'endereco', 'bairro', 'cep', 'zona', 'secao', 'eleitores'])

In [11]:
df_locais_votacao

,cod_municipio,municipio,cod_local,local,endereco,bairro,cep,zona,secao,eleitores
0,71072,SAO PAULO,1147,CENTRO DE CIENCIAS MATEMATICAS DA PUC,"RUA MARQUES DE PARANAGUA, 111CONSOLACAO ...",CENTRO,00000-000,001,0252,456
1,71072,SAO PAULO,1147,CENTRO DE CIENCIAS MATEMATICAS DA PUC,"RUA MARQUES DE PARANAGUA, 111CONSOLACAO ...",CENTRO,00000-000,001,0253,461
2,71072,SAO PAULO,1147,CENTRO DE CIENCIAS MATEMATICAS DA PUC,"RUA MARQUES DE PARANAGUA, 111CONSOLACAO ...",CENTRO,00000-000,001,0254,463
3,71072,SAO PAULO,1147,CENTRO DE CIENCIAS MATEMATICAS DA PUC,"RUA MARQUES DE PARANAGUA, 111CONSOLACAO ...",CENTRO,00000-000,001,0255,457
4,71072,SAO PAULO,1147,CENTRO DE CIENCIAS MATEMATICAS DA PUC,"RUA MARQUES DE PARANAGUA, 111CONSOLACAO ...",CENTRO,00000-000,001,0256,458
...,...,...,...,...,...,...,...,...,...,...
44828,71579,TABOAO DA SERRA,1082,EEPG. FRANCISCO D'AMICO,RUA ARLINDO GENARIO DE FREITAJARDIM SAPORITO ...,CENTRO,00000-000,416,0083,466
44829,71579,TABOAO DA SERRA,1082,EEPG. FRANCISCO D'AMICO,RUA ARLINDO GENARIO DE FREITAJARDIM SAPORITO ...,CENTRO,00000-000,416,0084,462
44830,71579,TABOAO DA SERRA,1082,EEPG. FRANCISCO D'AMICO,RUA ARLINDO GENARIO DE FREITAJARDIM SAPORITO ...,CENTRO,00000-000,416,0085,472
44831,71579,TABOAO DA SERRA,1082,EEPG. FRANCISCO D'AMICO,RUA ARLINDO GENARIO DE FREITAJARDIM SAPORITO ...,CENTRO,00000-000,416,0086,491


In [12]:
sum(valida_secoes)

44833

In [13]:
sum(valida_eleitores)

23309915

In [14]:
df_locais_votacao.eleitores.sum()

23309915

In [76]:
df_locais_votacao.to_csv('resultados/locais_de_votacao-1998.csv')